

# Nettoyage de la base de données

### <u>1. Description de la base de données

**Les données** </u>

La base de données utilisée est celle de Data.gouv qui liste les festivals en France. Les critères de sélection des festivals sont les suivants :
- avoir eu lieu en 2019
- avoir connu au moins deux éditions en 2019
- se dérouler sur plus d’une journée
- compter au moins 5 spectacles, représentations, concerts ou projections
<u> 



**Description de la base de données** </u>

La base de donnée regroupe les éléments suivants :
- Identité du festival : nom, disciplines dominantes, sous-catégorie au sein des disciplines
- Données géographiques : région de déroulement, département, adresse postale, géocodage xy (coordonnées)
- Données administratives : identifiant, code insee...
- Données temporelles : période de déroulement, date de création...
- Trouver/communiquer avec le festival : site internet et adresse mail.

### <u>2. Suppression des données inutiles 
</u>

On a choisit de supprimer les colonnes suivantes : 
- Code postal (de la commune principale de déroulement)
- Code Insee commune
- Code Insee EPCI
- Libellé EPCI	
- Numéro de voie	 
- Type de voie (rue, Avenue, boulevard, etc.)	
- Nom de la voie	
- Adresse postale	
- Complément d'adresse (facultatif)
- Décennie de création du festival
- Année de création du festival
- Identifiant Agence A
- identifiant CNM

On a aussi fait le choix de ne conserver que les festivals se déroulant en France métropolitaine. 


In [ ]:
import pandas as pd


# Charger le fichier CSV
df = pd.read_csv('festivals_en_France.csv', sep=';', encoding='utf-8-sig')

# Afficher les premières lignes pour vérifier le contenu
print(df.head())


# Supprimer les colonnes qui ne nous intéressent pas
df = df.drop(["Code postal (de la commune principale de déroulement)", "Code Insee commune", "Code Insee EPCI", "Libellé EPCI", "Numéro de voie", "Type de voie (rue, Avenue, boulevard, etc.)", "Nom de la voie", "Adresse postale", "Complément d'adresse (facultatif)", "Décennie de création du festival", "Année de création du festival", "Identifiant Agence A", "identifiant CNM"], axis=1)

# Afficher les colonnes restantes pour vérifier
print(df.columns.tolist())


# Supprimer les festivals hors France métropolitaine
regions_non_metropolitaines = ["Guadeloupe", "Martinique", "Guyane", "La Réunion", "Mayotte", "Polynésie française", "Saint-Pierre-et-Miquelon", "Saint-Barthélemy"]
df = df[~df["Région principale de déroulement"].isin(regions_non_metropolitaines)]

# Vérifier le contenu après suppression
print(df["Région principale de déroulement"].unique())


### <u>3. Modification et mise en forme des données  

**Transformation des données** </u>

Transformation des données sous forme de listes de listes. La première clef utilisée est nom du festival_identifiant car il y a des festivals ayant le même nom mais se déroulant sur des communes différentes. Ajouter l'identifiant permet d'avoir une clef unique. 

On veut quelque chose de cette forme : {"nom-festival_id" : {"nom-festival:..., "date":..., }}, {"nom-festival_id2" : {...} }, etc. 





In [ ]:
# Construire un dictionnaire où chaque clé principale est "Nom du festival_Identifiant"
dictionnaire = {}
for _, row in df.iterrows():
    # Utiliser la colonne "Identifiant" du DataFrame comme partie de la clé
    cle_principale = f"{row['Nom du festival']}_{row['Identifiant']}"
    
    # Ajouter un sous-dictionnaire contenant toutes les colonnes, sauf la colonne "Identifiant"
    dictionnaire[cle_principale] = row.drop("Identifiant").to_dict()

# Afficher un exemple
list(dictionnaire.items())[:5]  # Afficher les 5 premières entrées du dictionnaire pour vérifier


<u>

**La catégorie "pluridisciplinaire"**  </u>

Il s'agit ici d'expliciter la catégorie "pluridisciplinaire". En effet, l'utilisateur va cocher le type de festival qu'il souhaite : il faut donc spécifier à quelles disciplines appartient le festival pour que celui-ci apparaisse lorsque l'utilisateur cochera. Ainsi, si la colonne "Sous-catégorie [...]" n'est pas vide, on renomme la colonne "Discipline dominante" avec le nom de la sous-catégorie.

**Par exemple** : le festival "Rêves d'enfants" est noté pluridisciplinaire et est un festival de littérature et de spectacle vivant. Il faut qu'il apparaisse lorsque l'utilisateur coche "Littérature" ou "Spectacle vivant". On renomme ainsi la colonne "Discipline dominante" "Livre, littérature et Spectacle vivant". 


Il reste néanmoins 290 festivals marqués comme "pluridisciplinaire" mais sans sous-catégorie renseignée. Nous avons donc choisi de supprimer ces festivals pour deux raisons : 
1. Dans le questionnaire, l'utilisateur clique sur une catégorie de festival qu'il souhaite, si le festival n'en a pas il ne peut cliquer dessus. 
2. Il y a trop de festivals dans ce cas (290) pour pouvoir ajouter manuellement les catégories en question. 

**Par exemple**, le festival Printemps de paroles est noté pluridisciplinaire, mais avec aucune sous-catégorie associée.

In [ ]:
# Liste des colonnes sous-catégories que l'on analyse
colonnes_sous_categories = [
    "Sous-catégorie spectacle vivant",
    "Sous-catégorie musique",
    "Sous-catégorie cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques",
    "Sous-catégorie livre et littérature"
]

# Dictionnaire pour associer les colonnes de sous-catégorie à leurs noms dans "Discipline dominante"
mapping_discipline = {
    "Sous-catégorie spectacle vivant": "Spectacle vivant",
    "Sous-catégorie musique": "Musique",
    "Sous-catégorie cinéma et audiovisuel": "Cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques": "Arts visuels et numériques",
    "Sous-catégorie livre et littérature": "Livre et littérature"
}

# Fonction pour renommer la discipline dominante en respectant les correspondances spécifiques
def renommer_discipline(row):
    # Si "Discipline dominante" est pluridisciplinaire
    if row["Discipline dominante"] == "Pluridisciplinaire":
        # Liste des disciplines à partir des sous-catégories non vides
        sous_categories = [mapping_discipline[col] for col in colonnes_sous_categories if pd.notna(row[col])]
        # Si au moins une sous-catégorie est trouvée, les utiliser
        if sous_categories:
            return " et ".join(sous_categories)
        else:
            return "A supprimer"
    # Sinon, garder la valeur d'origine
    return row["Discipline dominante"]

# Appliquer la fonction sur le DataFrame
df["Discipline dominante"] = df.apply(renommer_discipline, axis=1)

# Supprimer les festivals marqués "A supprimer"
df = df[df["Discipline dominante"] != "A supprimer"]

# Vérifier les résultats
print(df[["Nom du festival", "Discipline dominante"]].head(10))

<u>

**Modification des sous-catégories** </u>

Pour plus de clarté, nous avons choisis de modifier les sous-catégories présentes dans la base de données qui étaient beaucoup trop nombreuses et les avons réduits à 10 par catégorie. 

In [ ]:
# Discipline dominante Spectacle vivant 

# Dictionnaire de regroupement des sous-catégories en 10 catégories principales
regroupements_spectacle_vivant = {
    "Théâtre": ["Théâtre", "Théâtre, Danse", "Théâtre, Marionnettes", "Théâtre, arts du conte"],
    "Danse": ["Danse contemporaine", "Danse traditionnelle", "Danse, théâtre"],
    "Arts de la Rue": ["Arts de la Rue", "Spectacles de rue", "Arts de la rue et Cirque"],
    "Cirque": ["Arts du Cirque", "Nouveau Cirque", "Arts de la piste"],
    "Musique et Chant": ["Musique", "Chanson", "Concerts", "Opéra"],
    "Marionnettes et Théâtre d'objets": ["Marionnettes", "Théâtre d'objets", "Marionnettes et Théâtre visuel"],
    "Spectacles pour Jeune Public": ["Jeune Public", "Spectacle pour enfants", "Conte musical"],
    "Performance et Arts Visuels": ["Performance", "Arts visuels", "Exposition"],
    "Humour et Café-Théâtre": ["Humour", "Café-Théâtre", "Théâtre d'humour"],
    "Pluridisciplinaire": ["Pluridisciplinaire", "Spectacle vivant pluridisciplinaire"]
}

# Inverser le dictionnaire pour faciliter la recherche de catégorie par sous-catégorie
inverse_regroupements = {}
for categorie, sous_cats in regroupements_spectacle_vivant.items():
    for sous_cat in sous_cats:
        inverse_regroupements[sous_cat] = categorie

# Fonction pour attribuer les nouvelles sous-catégories
def attribuer_sous_categories(row):
    sous_categories = str(row["Sous-catégorie spectacle vivant"]).split(", ")  # Diviser les sous-catégories multiples
    nouvelles_categories = set()
    
    for sous_cat in sous_categories:
        if sous_cat in inverse_regroupements:  # Vérifier si la sous-catégorie est dans le mapping
            nouvelles_categories.add(inverse_regroupements[sous_cat])
    
    return ", ".join(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction sur le DataFrame pour créer une nouvelle colonne "Nouvelles sous-catégories"
df["Nouvelles sous-catégories spectacle vivant"] = df.apply(attribuer_sous_categories, axis=1)

# Vérifier les résultats
#print(df[["Nom du festival", "Sous-catégorie spectacle vivant", "Nouvelles sous-catégories spectacle vivant"]].head(20))

ligne_128 = df.iloc[120]
print(ligne_128)